# LIBRARIES & FUNCTIONS

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect, Cross
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

import pydicom

import json

from Gafchromic import Radiochromic_RB
from Gafchromic_multichannel import Radiochromic_multilinear

import time


output_notebook()

Loading BokehJS ...

In [2]:
# FONCTIONS #




# Colors to be used in multiple lines graphs
colors = ['aqua', 'aquamarine', 'azure', 'bisque', 'black', 'blue', 'blueviolet', 'brown', 'burlywood', 
          'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan',
           'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkmagenta', 
          'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 
          'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 
          'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 
          'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'green', 'greenyellow', 'grey', 'honeydew', 'hotpink', 
          'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 
          'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgreen', 'lightgrey', 'lightpink', 
          'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 
          'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine', 'mediumblue', 
          'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise', 
          'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 
          'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 
          'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 
          'royalblue', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 
          'slateblue', 'slategray', 'slategrey', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 
          'turquoise', 'violet', 'wheat']



# fitting functions for RBGB model:
def funcDose(x, a, b, c, d, e):
    return a*(x[:,0]*x[:,1])+  b*(x[:,0]**c*x[:,1]**d) + e

def funcRBGB(x, a, b, c, d):
    return a*x*x*x +  b*x*x + c*x + d



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray




# Finds which model from the list of models corresponds best to the rb & gb values
#   
# @params:
#  rb : array of column R/B
#  gb : array of column G/B
#  modelsArr : list of parameters of the models of RBGB curves 
#  dispGraphs : displays graphs if true
def findBestModel(rb, gb, modelsArr, dispGraph=False):
    
    # calculates mean error between model and curve for all models
    errorValues = []
    for i in range(len(modelsArr)):
        mod = modelsArr[i]
        err = np.mean((gb-funcRBGB(rb, mod[0], mod[1], mod[2], mod[3]))*(gb-funcRBGB(rb, mod[0], mod[1], mod[2], mod[3])))
        errorValues.append(err)
    
    errormin = np.amin(errorValues)
    bestmodel = errorValues.index(errormin)
    
    if dispGraph:
        print("  >>> Best Model:", bestmodel)
        print("      Associated Error:", errormin)
        
        p1 = figure(plot_width=600, plot_height=600, title="RB vs GB", 
                    toolbar_location="above")

        p1.circle(rb, gb, size=3, color='cornflowerblue', alpha=0.3, 
                  legend_label="column = "+str(column)) 

        minrb = np.min(rb)
        maxrb = np.max(rb)
        xRBGB = np.arange(minrb, maxrb, (maxrb-minrb)/50)
        mod = modelsArr[bestmodel]
        p1.line(xRBGB, funcRBGB(xRBGB, mod[0], mod[1], mod[2], mod[3]), 
                line_color="black", line_width=1, line_dash='dotted')

        p1.xaxis.axis_label = "R/B values" 
        p1.yaxis.axis_label = "G/B values"
        p1.legend.location = 'top_left' 
        
        show(p1)
    
    
    return [bestmodel, errormin]




# Converts an image to dose using RBGB models
#   
# @params:
#  arr : image array
#  models : dictionary of RBGB model
def convertToDose(arr, models):
    
    rb = arr[:,:,0]/arr[:,:,2]
    gb = arr[:,:,1]/arr[:,:,2]
    
    doseImg = np.zeros(rb.shape)
    
    rbgbmodels = models["RBGBcurve"]
    for i in range(rb.shape[1]):
        [index, err] = findBestModel(rb[:,i], gb[:,i], rbgbmodels, dispGraph=False)
        
        rbgb = np.zeros((rb.shape[0],2))
        rbgb[:,0] = rb[:,i]
        rbgb[:,1] = gb[:,i]
        mod = models["DoseCurve"][index]
        doseImg[:,i] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])
    
    return doseImg




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    

    
# Displays two images and profiles
#   
# @params:
#  array: image array on which to perform streak correction.
def applyStreakCorrection(array):
        
        # First pixels sum and normalization:
        firstPixAvg_r = np.sum(array[0:10,:,0] ,axis=0)
        firstPixAvg_g = np.sum(array[0:10,:,1] ,axis=0)
        firstPixAvg_b = np.sum(array[0:10,:,2],axis=0)

        firstPixAvg_r = firstPixAvg_r / np.mean(firstPixAvg_r)
        firstPixAvg_g = firstPixAvg_g / np.mean(firstPixAvg_g)
        firstPixAvg_b = firstPixAvg_b / np.mean(firstPixAvg_b)


        # Last pixels sum and normalization:
        lastPixAvg_r = np.sum(array[-9:,:,0] ,axis=0)
        lastPixAvg_g = np.sum(array[-9:,:,1] ,axis=0)
        lastPixAvg_b = np.sum(array[-9:,:,2],axis=0)

        lastPixAvg_r = lastPixAvg_r / np.mean(lastPixAvg_r)
        lastPixAvg_g = lastPixAvg_g / np.mean(lastPixAvg_g)
        lastPixAvg_b = lastPixAvg_b / np.mean(lastPixAvg_b)


        # Streak correction image:
        #  This correction must be done on an uncroped image
        streakCorrImage = np.zeros(array.shape)
        x = [0, array.shape[1]-1]

        for i in range(array.shape[1]):
            allpix_y = range(array.shape[0])
            y0 = [firstPixAvg_r[i], lastPixAvg_r[i]]
            streakCorrImage[:,i,0] = 1 / np.interp(allpix_y, x, y0)
    
            y1 = [firstPixAvg_g[i], lastPixAvg_g[i]]
            streakCorrImage[:,i,1] = 1 / np.interp(allpix_y, x, y1)

            y2 = [firstPixAvg_b[i], lastPixAvg_b[i]]
            streakCorrImage[:,i,2] = 1 / np.interp(allpix_y, x, y2)
    
    
        # Apply streak correction to the image:
        return array * streakCorrImage

 


# Registers two images and return the moving img registered on the ref. img.
#   
# @params:
#  refImg: reference image
#  movingImg: moving image
def registerImgs(refImg, movingImg):

    # Initial transform:
    fixedimg = sitk.Image(refImg.shape[1], refImg.shape[0], sitk.sitkFloat32)
    for i in range(refImg.shape[1]):
        for j in range(refImg.shape[0]):
            fixedimg.SetPixel(i, j, float(refImg[j,i]))

    movingimg = sitk.Image(movingImg.shape[1], movingImg.shape[0], sitk.sitkFloat32)
    for i in range(movingImg.shape[1]):
        for j in range(movingImg.shape[0]):
            movingimg.SetPixel(i, j, float(movingImg[j,i]))

    initial_transform = sitk.CenteredTransformInitializer(fixedimg, 
                                                          movingimg, 
                                                          sitk.Euler2DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)

    
    # Real registration:
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings:
    registration_method.SetMetricAsMeanSquares()
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.2)
    registration_method.SetInterpolator(sitk.sitkLinear)


    # Optimizer settings.
    registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=1.0, minStep=0.01, 
                                                                 numberOfIterations=1000, relaxationFactor=0.5)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    final_transform = registration_method.Execute(sitk.Cast(fixedimg, sitk.sitkFloat32), 
                                                  sitk.Cast(movingimg, sitk.sitkFloat32))
    finalimg = sitk.Resample(movingimg, fixedimg, final_transform, sitk.sitkBSplineResamplerOrder5, 0.0, 
                             movingimg.GetPixelID())
    
    # Returns the resulting image:
    return sitk.GetArrayFromImage(finalimg)



# Conversion to dose:

In [90]:
# VARIABLES :

m_path = 'testRBGB/'
m_nbOfFiles = 1
m_firstNb = 1
m_GafFilesName = "testRBGB_00"
# m_GafFilesName = "ppp_127"
m_extension = ".tif"



m_subFactor = 10

m_RBGBFile = "C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_center.txt"
# m_RBGBFile2 = "C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_mean.txt"
m_RBGBFile2 = "C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_center&side_halfWidth3_meanDose.txt"

m_splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_normalCalib.txt'
m_multiLinearFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/multiLinearRegressionCoefficients_02282001.txt'




In [91]:
# READS THE GAF FILE:


visu = False

# Reads the first image:
img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb)+m_extension)

sizex = img.GetWidth()
sizey = img.GetHeight()
imgOrigin = img.GetOrigin()
imgSpacing = img.GetSpacing()


# Reads all images and does the median image:
size = (sitk.GetArrayFromImage(img).shape[0], 
        sitk.GetArrayFromImage(img).shape[1], 
        sitk.GetArrayFromImage(img).shape[2], 
        m_nbOfFiles)
imgs = np.zeros(size)

for i in range(m_nbOfFiles):
    img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb+i)+m_extension)
    imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    
array = np.median(imgs, axis=3)
# array = sitk.GetArrayFromImage(img)

array = subSampleRGBArray(array, m_subFactor)

# displays the image to select the area to convert:

if visu:
    p1 = figure(plot_width=600, plot_height=int(600*array.shape[0]/array.shape[1]), 
               title='Image to convert', toolbar_location="above")
    p1.image(image=[array[:,:,0]], x=0, y=0, dw=array.shape[1], dh=array.shape[0], palette="Plasma256")

    show(p1)


In [104]:
# Convert an image to dose using the models:



# Image to convert to dose: [y1:y2, x1:x2, :]
arrayToConvert = array[50:290, 50:260, :]

# Reading RBGB models:
with open(m_RBGBFile, 'r') as file:
    gafModelsDict = json.load(file)

with open(m_RBGBFile2, 'r') as file:
    gafModelsDict2 = json.load(file)

    
# conversion using the models:
doseimg_RBGB = convertToDose(arrayToConvert, gafModelsDict)

doseimg_RBGB2 = convertToDose(arrayToConvert, gafModelsDict2)

print("  > Image converted to dose using RBGB model.")


# conversion using classic R/B:
try:
    g = Radiochromic_RB(m_path+m_GafFilesName, m_firstNb, m_nbOfFiles)
    g.subSampleDataArray(10)
    a = g.cropImg(50,260,50,290)
    doseimg_basicRB = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    size = g.getSize()
except ValueError as err:
    print('Erreur: ' + err)

print("  > Image converted to dose using basic RB model.")

    
# conversion using RGB multilinear approach:
g = Radiochromic_multilinear(m_path+m_GafFilesName, m_firstNb, m_nbOfFiles)
g.subSampleDataArray(10)
g.readMultilinearRegressionFile(m_multiLinearFile, dispStatus=False)

ctrlRect_sub = [120,280,200,290]
toDoseRect_sub = [50,50,260,290]
    
doseimg_RGBmultilinear = g.convertToDose(toDoseRect_sub, ctrlRect_sub)
       
print("  > Image converted to dose using basic RGB multilinear model.")



  > Image converted to dose using RBGB model.
  > Image converted to dose using basic RB model.
  > Image converted to dose using basic RGB multilinear model.


In [107]:
# shows dose images:

compare2Imgs(doseimg_RBGB, doseimg_RGBmultilinear, 50, 50, title1="calibRBGB_center", title2="RGB multilinear")

# MULTIPLE IMAGE SET

In [109]:
# MULTIPLE IMAGES TEST R/B method:


# Variables:
path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/testLateral/'
nbOfFiles = 1
firstNb = 1
refImgName = "test-pos04_0"
fileExtension = ".tif"
splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_normalCalib.txt'
splineFileWithLatCorr = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_calibWithLateralCorr.txt'
pixsize = 1

otherImgNames = ["test-pos01_0", "test-pos02_0", "test-pos03_0", "test-pos05_0", 
                 "test-pos06_0", "test-pos07_0", "test-pos08_0"]

rect_ref = [95,110,245,160]

rect_imgs = [[0, 110, 140, 160],
            [25,110,165,160],
            [65,110,205,160],
            [125,110,265,160],
            [155,110,295,160],
            [160,110,300,160],
            [168,110,308,160]]

m_dosemax = 700

visu = False
subfactor = 10
dimViewer = 480
regImgNbToDisp = 6
line = 50
col = 50


# Reading the files:


try:
    # Opens reference image:
    img = sitk.ReadImage(path+refImgName+str(firstNb)+fileExtension)
    size = (sitk.GetArrayFromImage(img).shape[0], 
            sitk.GetArrayFromImage(img).shape[1], 
            sitk.GetArrayFromImage(img).shape[2], 
            nbOfFiles)

    imgs = np.zeros(size)
    for i in range(nbOfFiles):
        img = sitk.ReadImage(path+refImgName+str(firstNb+i)+fileExtension)
        imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    
    array = np.median(imgs, axis=3)
    array = subSampleRGBArray(array, subfactor)[rect_ref[1]:rect_ref[3],
                                                rect_ref[0]:rect_ref[2]]

    with open(m_RBGBFile, 'r') as file:
        gafModelsDict = json.load(file)
    
    refImg = convertToDose(array, gafModelsDict)

    
    # Opens all other images:
    imgs_lat = np.zeros([refImg.shape[0], refImg.shape[1], len(otherImgNames)])
    for i in range(len(otherImgNames)):
        img = sitk.ReadImage(path+otherImgNames[i]+str(m_firstNb)+fileExtension)
        size = (sitk.GetArrayFromImage(img).shape[0], 
                sitk.GetArrayFromImage(img).shape[1], 
                sitk.GetArrayFromImage(img).shape[2], 
                nbOfFiles)

        imgs = np.zeros(size)
        for j in range(nbOfFiles):
            img = sitk.ReadImage(path+otherImgNames[i]+str(firstNb+j)+fileExtension)
            imgs[:,:,:,j] = sitk.GetArrayFromImage(img)
    
        array = np.median(imgs, axis=3)
        array = subSampleRGBArray(array, subfactor)[rect_imgs[i][1]:rect_imgs[i][3],
                                                rect_imgs[i][0]:rect_imgs[i][2]]

        latImg = convertToDose(array, gafModelsDict)
        
        imgs_lat[:,:,i] = registerImgs(refImg, latImg)
    
except ValueError as err:
    print('Erreur: '+err)

    
    
# Visualization:    
if visu:
    img1 = refImg
    img2 = imgs_lat[:,:,regImgNbToDisp]
    imgsProfile = imgs_lat
    
    # Img 1:
    p1 = figure(plot_width=dimViewer, plot_height=int(dimViewer*img1.shape[0]/img1.shape[1]), 
                title='Reference image', toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=dimViewer, plot_height=int(dimViewer*img2.shape[0]/img2.shape[1]), 
               title='Registered image', toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=dimViewer, plot_height=int(dimViewer*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=3, line_color='black', legend_label='Reference Image')
    for i in range(len(otherImgNames)):
        p3.line(x3, imgsProfile[line,:,i], line_width=1, line_color=colors[i], legend_label='pos no'+ str(i))
    

    # Vertical profile:
    p4 = figure(plot_width=dimViewer, plot_height=int(dimViewer*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color='black', legend_label='Reference Image')
    for i in range(len(otherImgNames)):
        p4.line(x4, imgsProfile[:,col,i], line_width=2, line_color=colors[i], legend_label='pos no'+ str(i))

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)
    
print("  >>> mULTIPLE iMAGE tEST dONE !")

  >>> mULTIPLE iMAGE tEST dONE !


In [110]:
img1 = refImg
img2 = imgs_lat[:,:,1]
imgsProfile = imgs_lat

line = 25
col = 50

    # Img 1:
p1 = figure(plot_width=dimViewer, plot_height=int(dimViewer*img1.shape[0]/img1.shape[1]), 
             title='Reference image', toolbar_location="above")
p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
p2 = figure(plot_width=dimViewer, plot_height=int(dimViewer*img2.shape[0]/img2.shape[1]), 
            title='Registered image', toolbar_location="above")
p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
maxx = np.amax(img1[line,:])
if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
p3 = figure(plot_width=dimViewer, plot_height=int(dimViewer*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
x3 = np.arange(0, len(img1[line,:]), 1)
x3b = np.arange(0, len(img2[line,:]), 1)
p3.line(x3, img1[line,:], line_width=3, line_color='black', legend_label='Reference Image')
for i in range(len(otherImgNames)):
    p3.line(x3, imgsProfile[line,:,i], line_width=1, line_color=colors[i])#, legend_label='pos no'+ str(i))
    

    # Vertical profile:
p4 = figure(plot_width=dimViewer, plot_height=int(dimViewer*2/3), title="y profile", toolbar_location="above")
x4 = np.arange(0, len(img1[:,col]), 1)
x4b = np.arange(0, len(img2[:,col]), 1)
p4.line(x4, img1[:, col], line_width=3, line_color='black', legend_label='Reference Image')
for i in range(len(otherImgNames)):
    p4.line(x4, imgsProfile[:,col,i], line_width=2, line_color=colors[i])#, legend_label='pos no'+ str(i))

grid = gridplot([[p1, p2], [p3, p4]])


show(grid)
    


In [ ]:
# A FAIRE: COMPARAISON AVEC ETALONNAGE MULTI LINEAIRE, AJOUTER IMAGES AU MODELE POUR PLUS DE LIGNES DANS MODELE
#   VOIR AVEC AUTRE FILMS POUR SAVOIR SI CA FONCTIONNE ENCORE, VOIR POUR AUTRE RESOLUTION ET BRUIT...